In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

Повторяем действия из блокнота Train_Preporocessing с той лишь разницей, что мы будем делить времяные ряды на интервалы между событиями по колонке "pump_id" в связи с отсутствием целевых колонок.

In [18]:
data_raw=pd.read_csv('..//private_input.csv')
data_raw.head(3)

,pump_id,Well_Number,Date,Gas_Rate,Water_Rate,Gas_Capacity,Water_Capacity,Gas_Pressure,Pump_Pressure,Pump_Speed,Pump_Torque,Pump_Volumetric_Eff,Downhole_Gauge_Fluid_Level_Above_Sensor,Pump_Torque.1,PUMP_ID_PHASE
0,NaN,AGB005,2019-07-30,0.0,0.000000,0.0,0.009857,-6.89,0.00,0.0,NaN,0.000000,NaN,NaN,9.0
1,NaN,AGB005,2019-07-31,0.0,0.011606,0.0,0.009857,-6.89,723.95,58.0,51.52116,100.060345,NaN,NaN,9.0
2,NaN,AGB005,2019-08-01,0.0,0.013991,0.0,0.036567,-6.89,903.21,69.0,69.14682,101.391304,NaN,NaN,9.0


In [19]:
data_raw['pump_id'].notna().sum()

819

In [20]:
print(data_raw.dtypes)
numeric__indices = np.where(data_raw.dtypes == np.float)[0]
numeric__indices

pump_id                                    float64
Well_Number                                 object
Date                                        object
Gas_Rate                                   float64
Water_Rate                                 float64
Gas_Capacity                               float64
Water_Capacity                             float64
Gas_Pressure                               float64
Pump_Pressure                              float64
Pump_Speed                                 float64
Pump_Torque                                float64
Pump_Volumetric_Eff                        float64
Downhole_Gauge_Fluid_Level_Above_Sensor    float64
Pump_Torque.1                              float64
PUMP_ID_PHASE                              float64
dtype: object


array([ 0,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14], dtype=int64)

In [21]:
well_dict={w:pd.DataFrame() for w in data_raw['Well_Number'].unique()}
for i in data_raw['Well_Number'].unique():
       well_dict[i]=data_raw[data_raw['Well_Number']==i]

In [22]:
time_lag=[]
for i in well_dict.keys():
    time_lag.append(len(well_dict[i].iloc[:,0])) 
time_lag.sort()
min_time_lag=time_lag[0]
print(min_time_lag)

276


In [23]:
well_dict_ts={wts:pd.DataFrame() for wts in well_dict.keys()}
for i in well_dict.keys():
    well_dict_ts[i]=well_dict[i].copy()
    well_dict_ts[i]['Date'] = pd.to_datetime(well_dict[i]['Date'])
    well_dict_ts[i] = well_dict[i].set_index('Date')

In [24]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)

In [25]:
well_dict_ts_fill={wts:pd.DataFrame() for wts in well_dict_ts.keys()}
for i in well_dict_ts_fill.keys():
    not_fill_columns=well_dict_ts[i][['pump_id', 'Well_Number']].copy()
    well_ts=well_dict_ts[i].copy()
    well_ts.drop(['pump_id', 'Well_Number'],axis=1,inplace=True)
    well_fill=imputer.fit_transform(well_ts)
    if well_ts.shape!=well_fill.shape:
        dop=len(well_ts.iloc[0])-len(well_fill[0])
        well_fill=np.column_stack((well_fill,( np.zeros([len(well_fill[:,1]),dop]))))
    well_fill_df=pd.DataFrame(well_fill,columns=well_ts.columns,index=well_ts.index)
    well_dict_ts_fill[i]=not_fill_columns.join(well_fill_df)

In [26]:
def emergency_split(df,split_column):
    border=df.loc[df[split_column].notna()].index
    dict_spliten={k:pd.DataFrame() for k in range(len(border)+1)}
    for i in range(len(border)+1):
        if i ==0:
            dft=df[df.index<=border[i]]
            dict_spliten[i]=dft
        elif i==len(border):
            dft=df[df.index>border[i-1]]
            dict_spliten[i]=dft
        else:    
            dft=df[(df.index<=border[i])&(df.index>border[i-1])]
            dict_spliten[i]=dft
    
    return(dict_spliten)

In [27]:
def meaning(df, n_space=20):
    idxs=df.index
    interval=np.linspace(0,(len(df.iloc[:,0])-1),n_space,dtype=int)
    dfr=pd.DataFrame()
    c=0
    for i in interval:
        if i ==0:
            dft=df[(df.index<=idxs[i])]
            dfr[idxs[i]]=dft.mean(axis=0)
        elif i==len(interval)-1:
            dft=df[(df.index<=idxs[i])]
            dfr[idxs[i]]=dft.mean(axis=0)
        else:
            dft=df[(df.index<=idxs[i])]
            dfr[idxs[i]]=dft.mean(axis=0)
        c=i
    return dfr.T

In [28]:
def strach_limit(df,shift=50):
    if len(df.iloc[:,1])<shift:
        l=shift-len(df.iloc[:,1])
        zes=pd.DataFrame(np.zeros([l,len(df.iloc[1])]),columns=df.columns)
        result=zes.append(df,ignore_index=True)
    else:
        result=df.tail(shift)
    return result

In [29]:
well_dict_preproc={wtsf:pd.DataFrame() for wtsf in well_dict_ts_fill.keys()}
for i in well_dict_preproc.keys():
    if well_dict_ts_fill[i].loc[:,'pump_id'].notna().sum()!=0:
        sdw=emergency_split(well_dict_ts_fill[i],'pump_id')
        proc=pd.DataFrame()
        for k in sdw.keys():
            if len(sdw[k].iloc[:,0])!=0:
                sw_df=strach_limit(sdw[k],35)
                series_last=sdw[k][['pump_id', 'Well_Number']].tail(1)
                zi=sw_df.copy()
                zi.drop(['pump_id', 'Well_Number'],axis=1,inplace=True)
                zi=meaning(zi,10)
                s=pd.DataFrame(zi.values.flatten()).T
                s.index=series_last.index
                d=series_last.join(s)
                proc=proc.append(d)
        well_dict_preproc[i]=proc

In [30]:
preproc_data=pd.DataFrame()
for i in well_dict_preproc.keys():
    preproc_data=preproc_data.append(well_dict_preproc[i])
preproc_data

,pump_id,Well_Number,0,1,2,3,4,5,6,7,...,110,111,112,113,114,115,116,117,118,119
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-05,9.0,AGB005,0.000000,0.000029,310.362,0.035931,3806.00,3803.57,0.0,-0.923731,...,303.096000,0.036022,2032.332286,2007.039143,0.000000,-0.501751,0.000000,282.307400,250.442286,9.0
2021-01-27,NaN,AGB005,395.708679,0.081088,418.314,0.033546,1503.16,1129.91,258.0,241.250044,...,492.783086,0.030603,1500.746571,1156.640000,258.000000,241.634117,156.338291,172.895689,258.290571,9.0
2017-12-10,13.0,AGQ001,0.000000,0.000789,171.270,0.011924,3021.00,12.00,0.0,0.000000,...,172.456286,0.011833,2262.000000,11.000000,0.000000,0.000000,0.000000,587.225764,0.000000,13.0
2019-06-17,14.0,AGQ001,0.000000,0.000929,198.258,0.011447,298.00,135.58,0.0,0.000000,...,198.762171,0.011447,203.171429,93.316000,0.000000,0.000000,0.000000,879.695135,0.000000,14.0
2021-01-27,NaN,AGQ001,0.000000,0.000737,250.158,0.008426,308.97,0.71,0.0,0.000000,...,254.161714,0.008181,294.120571,0.879429,0.000000,0.000000,0.000000,691.385564,0.000000,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-26,3743.0,ZUP207,0.014976,0.003596,45.672,0.061687,163.00,178.59,240.0,161.000244,...,144.356143,0.058242,324.242000,100.011429,36.571429,24.657180,1.818446,447.310420,0.000000,3743.0
2019-11-23,3744.0,ZUP207,4.841115,0.001006,1239.372,0.043086,162.00,2.66,0.0,0.000000,...,448.801543,0.056041,170.942857,2.226000,0.000000,0.000000,0.000000,507.868471,0.000000,3744.0
2021-01-27,NaN,ZUP207,0.001720,0.001755,133.902,0.054533,166.27,2.17,0.0,0.000000,...,164.330229,0.052448,165.694857,2.288000,0.000000,0.000000,0.000000,513.978509,0.000000,3743.5


In [31]:
preproc_data.dropna()

,pump_id,Well_Number,0,1,2,3,4,5,6,7,...,110,111,112,113,114,115,116,117,118,119
Date,,,,,,,,,,,,,,,,,,,,,
2020-07-05,9.0,AGB005,0.000000,0.000029,310.362,0.035931,3806.00,3803.57,0.0,-0.923731,...,303.096000,0.036022,2032.332286,2007.039143,0.000000,-0.501751,0.000000,282.307400,250.442286,9.0
2017-12-10,13.0,AGQ001,0.000000,0.000789,171.270,0.011924,3021.00,12.00,0.0,0.000000,...,172.456286,0.011833,2262.000000,11.000000,0.000000,0.000000,0.000000,587.225764,0.000000,13.0
2019-06-17,14.0,AGQ001,0.000000,0.000929,198.258,0.011447,298.00,135.58,0.0,0.000000,...,198.762171,0.011447,203.171429,93.316000,0.000000,0.000000,0.000000,879.695135,0.000000,14.0
2017-05-07,33.0,AGQ007,0.000000,0.000395,213.828,0.025597,4.00,1.00,0.0,0.000000,...,206.562000,0.025724,3.857143,1.000000,0.000000,0.000000,0.000000,65.232353,254.425000,33.0
2020-05-09,34.0,AGQ007,0.000000,0.000213,164.004,0.024802,0.00,0.64,0.0,0.000000,...,164.004000,0.024802,0.600000,0.321714,0.000000,0.000000,0.000000,65.232353,254.425000,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-09,3720.0,ZUP197,0.000000,0.000299,187.878,0.020032,225.00,198.67,1.0,3.000005,...,188.678743,0.019991,193.998571,189.791143,0.857143,2.563193,0.000000,328.787124,271.421143,3720.0
2018-10-08,3730.0,ZUP201,471.397321,0.015710,538.722,0.056917,224.00,237.47,328.0,79.000120,...,521.357743,0.057283,219.142857,227.337714,264.028571,58.828660,14.305399,136.964139,273.949286,3730.0
2019-07-26,3743.0,ZUP207,0.014976,0.003596,45.672,0.061687,163.00,178.59,240.0,161.000244,...,144.356143,0.058242,324.242000,100.011429,36.571429,24.657180,1.818446,447.310420,0.000000,3743.0


In [33]:
preproc_data.dropna().to_csv('preprocessed_test.csv')